In [1]:
import numpy as np
import skimage.io as io
import skimage.segmentation
import os

Prior to running this notebook, perform adaptive cell segmentation in inForm using DAPI as a nuclear marker. Export nuclear label arrays in inForm as tiff files and move them to a folder (in_dir). This notebook is used for post-processing of nuclear label arrays from inForm to produce binary masks that can be used for evaluation and comparison with inForm with other nuclear segmentation platforms.

## Post Processing 
Post-processing of predicted label masks into binary masks for evaluation. The overall effect of this step is to separate touching nuclei by adding a 2-pixel gap (one-pixel erosion on each nucleus). Next, binary masks (both segmented and ground truth data) can be used to compute the F1-scores by utilizing connected component analysis. 

In [2]:
def postprocess(save_dir_seg, save_dir_bin):
    '''
    Produces binary masks for evaluation from the labelled mask output from the inForm

    Parameters
    save_dir_seg: str
        the path to the directory containing the label masks output by inForm. All images should be in .tiff format
    save_dir_bin: str
        the path to the directory where the binary images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    # iterating over all files in the input directory
    for file in os.listdir(save_dir_seg):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # Finding the pixels that are touching between any two nuclei
            label_mask = io.imread(os.path.join(save_dir_seg, file))
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir_bin, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

## Running the postprocessing pipeline

In [3]:
# Replace the strings for save_dir_seg and save_dir_bin to paths on the local machine
save_dir_seg = r'C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\inForm_label_masks'
save_dir_bin = r'C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\inForm_binary_masks'
# To create binary masks for inForm
postprocess(save_dir_seg,save_dir_bin)